In [9]:
from Spreadsheet.Spreadsheet import Spreadsheet

spreadsheet = Spreadsheet("pito.s2v")

spreadsheet.set("A1", "Hello") 

spreadsheet.set("A2", "2")

print(spreadsheet.get("A1"))

print(spreadsheet.get("A2"))


Hello
2.0


In [2]:
import re

class Tokenizer:
    def __init__(self, formula):
        self.formula = formula

    def tokenize(self):
        # Regular expression patterns for different token types
        pattern = {
            'operator': r'\+|-|\*|/',
            'coordinate': r'[A-Z]+\d+',
            'number': r'\d+\.?\d*',
            'opening_round_bracket': r'\(',
            'closing_round_bracket': r'\)',
            'colon': r':',
            'semi_colon': r';',
            'function': r'SUMA|PROMEDIO|MAX|MIN',  # Additional functions can be added as necessary
            'range': r'[A-Z]+\d+:[A-Z]+\d+'  # For example, A1:B3
        }

        # Combine the patterns into a single regular expression
        combined_pattern = '|'.join(f'({p})' for p in pattern.values())
        
        # Find all matches of the pattern in the formula
        tokens = re.findall(combined_pattern, self.formula)
        
        # Flatten the list of tuples and filter out empty strings
        tokens = [token for token in sum(tokens, ()) if token != '']

        return tokens

# Example usage
formula = "=1 + A1*((SUMA(A2:B5;PROMEDIO(B6:D8);C1;27)/4)+(D6-D8))"
tokenizer = Tokenizer(formula)
print(tokenizer.tokenize())


['1', '+', 'A1', '*', '(', '(', 'SUMA', '(', 'A2', ':', 'B5', ';', 'PROMEDIO', '(', 'B6', ':', 'D8', ')', ';', 'C1', ';', '27', ')', '/', '4', ')', '+', '(', 'D6', '-', 'D8', ')', ')']


In [3]:
import re

class Tokenizer:
    def __init__(self, formula):
        self.formula = formula

    def tokenize(self):
        # Regular expression patterns for different token types
        pattern = {
            'operator': r'\+|-|\*|/',
            'coordinate': r'[A-Z]+\d+',
            'number': r'\d+\.?\d*',
            'opening_round_bracket': r'\(',
            'closing_round_bracket': r'\)',
            'colon': r':',
            'semi_colon': r';',
            'function': r'SUMA|PROMEDIO|MAX|MIN',  # Additional functions can be added as necessary
            'range': r'[A-Z]+\d+:[A-Z]+\d+'  # For example, A1:B3
        }

        # Combine the patterns into a single regular expression
        combined_pattern = '|'.join(f'({p})' for p in pattern.values())
        
        # Find all matches of the pattern in the formula
        tokens = re.findall(combined_pattern, self.formula)
        
        # Flatten the list of tuples and filter out empty strings
        tokens = [token for token in sum(tokens, ()) if token != '']

        return tokens
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_index = 0
        self.function_stack = []  # Stack to track function context

    def next_token(self):
        if self.current_index < len(self.tokens):
            token = self.tokens[self.current_index]
            self.current_index += 1
            return token
        else:
            return None

    def parse(self):
        if not self.tokens:
            raise ValueError("No tokens to parse")

        last_token_type = None
        parenthesis_count = 0
        expecting_range = False  # Flag to track if a colon for a range is expected

        while self.current_index < len(self.tokens):
            token = self.next_token()
            #print(token)
            if self.is_function(token):
                if last_token_type in ['operand', 'closing_round_bracket']:
                    raise SyntaxError("Function name cannot follow an operand or closing bracket directly")
                inside_function = True
                self.function_stack.append(token)
                last_token_type = 'function'
            elif self.is_coordinate(token):
                # Handle cell coordinates
                if last_token_type == 'operand' and not expecting_range:
                    raise SyntaxError("Two consecutive operands")
                last_token_type = 'operand'
                expecting_range = True  # After a coordinate, a colon for range is valid
            elif token == ';':
                # Handle semicolon as function argument separator
                if not self.function_stack or last_token_type in ['operator', 'opening_round_bracket', ';']:
                    raise SyntaxError("Invalid use of ';' as argument separator")
                last_token_type = ';'
            elif self.is_number(token):
                # Handle numbers (including decimals)
                if last_token_type in ['operand', 'closing_round_bracket']:
                    raise SyntaxError("Invalid syntax: number follows an operand or closing bracket without an operator")
                last_token_type = 'operand'
                expecting_range = False
            elif token == ':':
                # Handle colon for cell ranges
                if not expecting_range or last_token_type != 'operand':
                    raise SyntaxError("Invalid use of ':' for cell range")
                last_token_type = 'range'
                expecting_range = False  # Reset the flag as colon is consumed
            elif token in '+-*/':
                if last_token_type in [None, 'operator', '(']:
                    raise SyntaxError("Operator in invalid position")
                last_token_type = 'operator'
            elif token == '(':
                parenthesis_count += 1
                if last_token_type in ['operand', 'closing_round_bracket']:
                    raise SyntaxError("Operand or closing bracket followed directly by '('")
                last_token_type = '('
            elif token == ')':
                parenthesis_count -= 1
                if last_token_type in ['operator', '(']:
                    raise SyntaxError("')' cannot follow an operator or '('")
                last_token_type = ')'
                if self.function_stack:
                    self.function_stack.pop()
            else:
                raise SyntaxError(f"Unknown token: {token}")

            if parenthesis_count < 0:
                raise SyntaxError("Unbalanced parentheses")

        if parenthesis_count != 0:
            raise SyntaxError("Unbalanced parentheses")

        return self.tokens

    def is_coordinate(self, token):
        # A cell coordinate is defined as one or more uppercase letters followed by one or more digits
        coordinate_pattern = r'^[A-Z]+\d+$'
        #print(re.match(coordinate_pattern, token) is not None)
        return re.match(coordinate_pattern, token) is not None

    def is_function(self, token):
        # List of valid function names
        valid_functions = ["SUMA", "PROMEDIO", "MAX", "MIN"]
        return token in valid_functions
    
    def is_number(self, token):
        number_pattern = r'^\d+\.?\d*$'
        return re.match(number_pattern, token) is not None

# Example usage
    
# Example usage
formula = "=1.05 + A1*((SUMA(A2:B5;PROMEDIO(B6:D8);C1;27)/4)+(D6-D8))"
tokenizer = Tokenizer(formula)
tokens = tokenizer.tokenize()
print(tokens)
parser = Parser(tokens)
try:
    parsed_tokens = parser.parse()
    print("Syntax is correct:", parsed_tokens)
except SyntaxError as e:
    print("Syntax error:", e)

['1.05', '+', 'A1', '*', '(', '(', 'SUMA', '(', 'A2', ':', 'B5', ';', 'PROMEDIO', '(', 'B6', ':', 'D8', ')', ';', 'C1', ';', '27', ')', '/', '4', ')', '+', '(', 'D6', '-', 'D8', ')', ')']
Syntax is correct: ['1.05', '+', 'A1', '*', '(', '(', 'SUMA', '(', 'A2', ':', 'B5', ';', 'PROMEDIO', '(', 'B6', ':', 'D8', ')', ';', 'C1', ';', '27', ')', '/', '4', ')', '+', '(', 'D6', '-', 'D8', ')', ')']


In [4]:
class GeneratePostfix:
    def __init__(self, tokens):
        self.tokens = tokens
        self.output_queue = []
        self.operator_stack = []
        # Update the precedence for ':' and ';'
        self.operators = {'+': 1, '-': 1, '*': 2, '/': 2, ':': 10, ';': 1}
        self.functions = {'SUMA', 'PROMEDIO', 'MAX', 'MIN'}

    def precedence(self, op):
        return self.operators.get(op, 0)

    def generate_postfix(self):
        for token in self.tokens:
            if self.is_number(token) or self.is_coordinate(token):
                self.output_queue.append(token)
            elif token in self.operators:
                while self.operator_stack and self.operator_stack[-1] != '(' and \
                      self.precedence(self.operator_stack[-1]) >= self.precedence(token):
                    self.output_queue.append(self.operator_stack.pop())
                self.operator_stack.append(token)
            elif token in self.functions:
                self.operator_stack.append(token)
            elif token == '(':
                self.operator_stack.append(token)
            elif token == ')':
                while self.operator_stack and self.operator_stack[-1] != '(':
                    self.output_queue.append(self.operator_stack.pop())
                self.operator_stack.pop()  # Pop '(' from stack
                if self.operator_stack and self.operator_stack[-1] in self.functions:
                    self.output_queue.append(self.operator_stack.pop())

        while self.operator_stack:
            self.output_queue.append(self.operator_stack.pop())

        return self.output_queue

    def is_coordinate(self, token):
        # A cell coordinate is defined as one or more uppercase letters followed by one or more digits
        coordinate_pattern = r'^[A-Z]+\d+$'
        #print(re.match(coordinate_pattern, token) is not None)
        return re.match(coordinate_pattern, token) is not None
    
    def is_number(self, token):
        number_pattern = r'^\d+\.?\d*$'
        return re.match(number_pattern, token) is not None

# Example usage
formula = "A4-B2/SUMA(A2:A4;B4)"
tokenizer = Tokenizer(formula)
tokens = tokenizer.tokenize()
print(tokens)
parser = Parser(tokens)
#formula_tokens = ['1.05', '+', 'A1', '*', '(', '(', 'SUMA', '(', 'A2', ':', 'B5', ';', 'PROMEDIO', '(', 'B6', ':', 'D8', ')', ';', 'C1', ';', '27', ')', '/', '4', ')', '+', '(', 'D6', '-', 'D8', ')', ')']
gen_postfix = GeneratePostfix(tokens)
postfix_expression = gen_postfix.generate_postfix()
print(postfix_expression)

# # Example usage
# formula = "=1.05 + A1*((SUMA(A2:B5;PROMEDIO(B6:D8);C1;27)/4)+(D6-D8))"
# tokenizer = Tokenizer(formula)
# tokens = tokenizer.tokenize()
# print(tokens)
# parser = Parser(tokens)
# try:
#     parsed_tokens = parser.parse()
#     print("Syntax is correct:", parsed_tokens)
# except SyntaxError as e:
#     print("Syntax error:", e)
# generator = GeneratePostfix(tokens)
# postfix_expression = generator.shunting_yard()
# print("Postfix Expression:", postfix_expression)



['A4', '-', 'B2', '/', 'SUMA', '(', 'A2', ':', 'A4', ';', 'B4', ')']
['A4', 'B2', 'A2', 'A4', ':', 'B4', ';', 'SUMA', '/', '-']


In [13]:
from Formula.Process.Parser import Parser
from Formula.Process.GeneratePostfix import GeneratePostfix
from Formula.Process.Tokenizer import Tokenizer
from Spreadsheet.Spreadsheet import Spreadsheet
import re

class EvaluatePostfix:
    def __init__(self, formula, Spreadsheet):
        self.formula = formula
        self.Spreadsheet = Spreadsheet
        self.postfix_expression = self.generate_postfix()

    def generate_postfix(self):
        tokenizer = Tokenizer(self.formula)
        tokens = tokenizer.tokenize()
        parser = Parser(tokens)
        tokens = parser.parse()
        postfix_generator = GeneratePostfix(tokens)
        postfix_expression = postfix_generator.generate_postfix()
        return postfix_expression

    def evaluate(self):
        stack = []
        function_args = []  # To accumulate arguments for functions

        for token in self.postfix_expression:
            if self.is_number(token):
                stack.append(float(token))
            elif self.is_coordinate(token):
                stack.append(token)
            elif token in ['+', '-', '*', '/']:
                if len(stack) < 2:
                    raise ValueError("Insufficient operands for operator")
                operand2 = stack.pop()
                operand1 = stack.pop()
                result = self.apply_operator(operand1, operand2, token)
                stack.append(result)
            elif token in ['SUMA', 'PROMEDIO', 'MAX', 'MIN']:
                if not function_args:  # If no arguments accumulated, raise an error
                    raise ValueError("No arguments provided for function")
                in_function = True
                result = self.apply_function(function_args, token)
                stack.append(result)
                function_args = []  # Clear arguments after processing
            elif token == ';':
                # Accumulate arguments for a function in reverse order
                temp_args = []
                while stack and stack[-1] != ':':
                    temp_args.append(stack.pop())
                if stack: stack.pop()  # Remove ':' from stack
                function_args.extend(reversed(temp_args))
            else:
                raise ValueError(f"Unrecognized token: {token}")
            print(stack)
            print(function_args)
        if len(stack) != 1:
            raise ValueError("Invalid Postfix Expression")
        
        return stack.pop()

    def apply_function(self, function_args, function):
        if function == 'SUMA':
            return sum(function_args)
        elif function == 'PROMEDIO':
            return sum(function_args) / len(function_args) if function_args else 0
        elif function == 'MAX':
            return max(function_args)
        elif function == 'MIN':
            return min(function_args)
        else:
            raise ValueError(f"Unrecognized function: {function}")



    def is_number(self, token):
        try:
            float(token)
            return True
        except ValueError:
            return False

    def is_coordinate(self, token):
        return re.match(r'^[A-Z]+\d+$', token) is not None

    def get_value_from_coordinate(self, coordinate):
        # Placeholder for getting value from a coordinate like 'A4'
        cell = self.Spreadsheet.get(coordinate)
        return cell.get_content().get_value()
        

    def apply_operator(self, operand1, operand2, operator):
        if operator == '+':
            return operand1 + operand2
        elif operator == '-':
            return operand1 - operand2
        elif operator == '*':
            return operand1 * operand2
        elif operator == '/':
            return operand1 / operand2
        else:
            raise ValueError(f"Unrecognized operator: {operator}")

    
# Example usage:
        
spreadsheet = Spreadsheet("pito.s2v")

spreadsheet.set("A1", "Hello")

spreadsheet.set("A2", "2")

print(spreadsheet.get("A1"))

print(spreadsheet.get("A2"))
postfix_expr = ['A2', '1', 'A2', 'A2', ';', 'SUMA', '/', '-']
formula= '1-SUMA(A2;A2)'
evaluator = EvaluatePostfix(formula, spreadsheet)
result = evaluator.evaluate()
print(result)

Hello
2.0
[1.0]
[]
[1.0, 2.0]
[]
[1.0, 2.0, 2.0]
[]
[]
[1.0, 2.0, 2.0]
[5.0]
[]


ValueError: Insufficient operands for operator